In [2]:
import tkinter as tk
import random
import math
from collections import deque

# ---------------------------
# Echo Dash (Tkinter)
# ---------------------------

WIDTH, HEIGHT = 800, 560
FPS = 60
DT = 1 / FPS
PLAYER_SPEED = 220  # pixels per second
PLAYER_R = 12
ECHO_R = 10
ORB_R = 8
ENEMY_R = 12
NUM_ORBS = 10
NUM_ENEMIES = 5
ECHO_DELAY_SEC = 1.6  # how long your shadow lags behind you
BORDER = 12

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def dist2(ax, ay, bx, by):
    dx, dy = ax - bx, ay - by
    return dx*dx + dy*dy

def circle_overlap(ax, ay, ar, bx, by, br):
    return dist2(ax, ay, bx, by) <= (ar + br) ** 2

class EchoDash:
    def __init__(self, root):
        self.root = root
        root.title("Echo Dash — Survive your own echo!")
        root.resizable(False, False)
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="#0b1021", highlightthickness=0)
        self.canvas.pack()

        self.keys = set()
        self.running = True
        self.game_over = False
        self.win = False
        self.time_accum = 0.0

        # HUD
        self.hud = self.canvas.create_text(
            10, 10, anchor="nw", fill="#cfe3ff",
            font=("Consolas", 14, "bold"),
            text=""
        )

        # Border
        self.canvas.create_rectangle(BORDER, BORDER, WIDTH - BORDER, HEIGHT - BORDER, outline="#415a77")

        # Init game entities
        self.reset_game()

        # Bindings
        self.root.bind("<KeyPress>", self.on_key_down)
        self.root.bind("<KeyRelease>", self.on_key_up)

        # Start loop
        self.tick()

    def reset_game(self):
        self.canvas.delete("entity")
        self.game_over = False
        self.win = False
        self.time_elapsed = 0.0

        # Player start
        self.px, self.py = WIDTH * 0.2, HEIGHT * 0.5
        self.vx, self.vy = 0.0, 0.0
        self.player = self.canvas.create_oval(
            self.px-PLAYER_R, self.py-PLAYER_R, self.px+PLAYER_R, self.py+PLAYER_R,
            fill="#62d2ff", outline="", tags=("entity", "player")
        )

        # Echo setup: record history of player positions; echo lags behind
        delay_frames = max(1, int(ECHO_DELAY_SEC * FPS))
        self.history = deque([(self.px, self.py)] * delay_frames, maxlen=delay_frames)
        hx, hy = self.history[0]
        self.ex, self.ey = hx, hy
        self.echo = self.canvas.create_oval(
            self.ex-ECHO_R, self.ey-ECHO_R, self.ex+ECHO_R, self.ey+ECHO_R,
            fill="#a5adbb", outline="", tags=("entity", "echo")
        )

        # Orbs
        self.orbs = []
        attempts = 0
        while len(self.orbs) < NUM_ORBS and attempts < 1000:
            attempts += 1
            ox = random.randint(BORDER+30, WIDTH-BORDER-30)
            oy = random.randint(BORDER+30, HEIGHT-BORDER-30)
            # keep away from player start
            if dist2(ox, oy, self.px, self.py) < (PLAYER_R + 120) ** 2:
                continue
            # keep orbs apart
            if any(dist2(ox, oy, x, y) < (ORB_R*2 + 20) ** 2 for (x, y, _) in self.orbs):
                continue
            orb_id = self.canvas.create_oval(
                ox-ORB_R, oy-ORB_R, ox+ORB_R, oy+ORB_R,
                fill="#5af78e", outline="", tags=("entity", "orb")
            )
            self.orbs.append((ox, oy, orb_id))

        # Enemies (bouncing)
        self.enemies = []
        for _ in range(NUM_ENEMIES):
            ex = random.randint(BORDER+40, WIDTH-BORDER-40)
            ey = random.randint(BORDER+40, HEIGHT-BORDER-40)
            # avoid immediate spawn kill
            if dist2(ex, ey, self.px, self.py) < (PLAYER_R + 160) ** 2:
                ex = WIDTH * 0.8
                ey = HEIGHT * 0.3
            speed = random.uniform(90, 160)
            angle = random.uniform(0, math.tau)
            evx = math.cos(angle) * speed
            evy = math.sin(angle) * speed
            eid = self.canvas.create_oval(
                ex-ENEMY_R, ey-ENEMY_R, ex+ENEMY_R, ey+ENEMY_R,
                fill="#ff6b6b", outline="", tags=("entity", "enemy")
            )
            self.enemies.append([ex, ey, evx, evy, eid])

        # Floating tip text
        self.tip = self.canvas.create_text(
            WIDTH//2, HEIGHT-26,
            fill="#9bbcff", font=("Consolas", 12),
            text="Move: Arrow Keys / WASD   Pause: P   Restart: R   Goal: Collect all orbs, avoid red and your echo.",
            tags=("entity",)
        )

    # ---------------------------
    # Input
    # ---------------------------
    def on_key_down(self, e):
        k = e.keysym.lower()
        self.keys.add(k)
        if k == "p":
            self.running = not self.running
        if k == "r":
            self.reset_game()

    def on_key_up(self, e):
        k = e.keysym.lower()
        self.keys.discard(k)

    def input_vector(self):
        left = ("left" in self.keys) or ("a" in self.keys)
        right = ("right" in self.keys) or ("d" in self.keys)
        up = ("up" in self.keys) or ("w" in self.keys)
        down = ("down" in self.keys) or ("s" in self.keys)
        x = (1 if right else 0) - (1 if left else 0)
        y = (1 if down else 0) - (1 if up else 0)
        if x == 0 and y == 0:
            return 0.0, 0.0
        mag = math.hypot(x, y)
        return x/mag, y/mag

    # ---------------------------
    # Game Loop
    # ---------------------------
    def tick(self):
        if self.running and not self.game_over and not self.win:
            self.update(DT)
        self.root.after(int(1000 / FPS), self.tick)

    def update(self, dt):
        self.time_elapsed += dt

        # Move player
        ix, iy = self.input_vector()
        self.vx = ix * PLAYER_SPEED
        self.vy = iy * PLAYER_SPEED
        self.px += self.vx * dt
        self.py += self.vy * dt

        # Keep inside bounds
        self.px = clamp(self.px, BORDER + PLAYER_R, WIDTH - BORDER - PLAYER_R)
        self.py = clamp(self.py, BORDER + PLAYER_R, HEIGHT - BORDER - PLAYER_R)

        # Update echo position from history
        self.history.append((self.px, self.py))
        hx, hy = self.history[0]
        self.ex, self.ey = hx, hy

        # Move enemies & bounce
        for e in self.enemies:
            e[0] += e[2] * dt
            e[1] += e[3] * dt
            if e[0] <= BORDER + ENEMY_R or e[0] >= WIDTH - BORDER - ENEMY_R:
                e[2] *= -1
                e[0] = clamp(e[0], BORDER + ENEMY_R, WIDTH - BORDER - ENEMY_R)
            if e[1] <= BORDER + ENEMY_R or e[1] >= HEIGHT - BORDER - ENEMY_R:
                e[3] *= -1
                e[1] = clamp(e[1], BORDER + ENEMY_R, HEIGHT - BORDER - ENEMY_R)

        # Collisions: enemies
        for ex, ey, evx, evy, eid in self.enemies:
            if circle_overlap(self.px, self.py, PLAYER_R, ex, ey, ENEMY_R):
                self.lose("You hit an enemy!")
                break

        # Collisions: echo (your past self)
        if not self.game_over and circle_overlap(self.px, self.py, PLAYER_R, self.ex, self.ey, ECHO_R):
            self.lose("You collided with your echo!")

        # Collect orbs
        if not self.game_over:
            remaining = []
            for ox, oy, oid in self.orbs:
                if circle_overlap(self.px, self.py, PLAYER_R, ox, oy, ORB_R):
                    self.canvas.delete(oid)
                else:
                    remaining.append((ox, oy, oid))
            self.orbs = remaining
            if not self.orbs:
                self.win = True
                self.show_banner("You collected all orbs! ✨\nPress R to play again.")

        # Draw updates
        self.redraw()

    def lose(self, reason):
        self.game_over = True
        self.show_banner(f"Game Over 💥\n{reason}\nPress R to restart.")

    def show_banner(self, text):
        # Semi-transparent overlay
        overlay = self.canvas.create_rectangle(0, 0, WIDTH, HEIGHT, fill="#000000", stipple="gray50", outline="", tags=("entity",))
        self.canvas.tag_lower(overlay)
        self.canvas.tag_raise(self.hud)
        self.canvas.create_text(
            WIDTH//2, HEIGHT//2, text=text, fill="#ffffff",
            font=("Consolas", 20, "bold"), justify="center", tags=("entity",)
        )

    def redraw(self):
        # Player
        self.canvas.coords(self.player, self.px-PLAYER_R, self.py-PLAYER_R, self.px+PLAYER_R, self.py+PLAYER_R)
        # Echo
        self.canvas.coords(self.echo, self.ex-ECHO_R, self.ey-ECHO_R, self.ex+ECHO_R, self.ey+ECHO_R)
        # Enemies
        for ex, ey, evx, evy, eid in self.enemies:
            self.canvas.coords(eid, ex-ENEMY_R, ey-ENEMY_R, ex+ENEMY_R, ey+ENEMY_R)

        # HUD
        orb_count = len(self.orbs)
        status = "Paused" if not self.running else "Playing"
        if self.game_over:
            status = "Game Over"
        elif self.win:
            status = "You Win!"
        self.canvas.itemconfig(
            self.hud,
            text=f"Status: {status}    Orbs left: {orb_count}    Time: {self.time_elapsed:4.1f}s"
        )

def main():
    root = tk.Tk()
    game = EchoDash(root)
    root.mainloop()

if __name__ == "__main__":
    main()


In [3]:
import tkinter as tk
import random
import math
from collections import deque

# ---------------------------
# Echo Dash (Tkinter)
# ---------------------------

WIDTH, HEIGHT = 800, 560
FPS = 60
DT = 1 / FPS
PLAYER_SPEED = 220  # pixels per second
PLAYER_R = 12
ECHO_R = 10
ORB_R = 8
ENEMY_R = 12
NUM_ORBS = 10
NUM_ENEMIES = 5
ECHO_DELAY_SEC = 1.6  # how long your shadow lags behind you
BORDER = 12


def clamp(v, lo, hi):
    return max(lo, min(hi, v))


def dist2(ax, ay, bx, by):
    dx, dy = ax - bx, ay - by
    return dx * dx + dy * dy


def circle_overlap(ax, ay, ar, bx, by, br):
    return dist2(ax, ay, bx, by) <= (ar + br) ** 2


class EchoDash:
    def __init__(self, root):
        self.root = root
        root.title("Echo Dash — Survive your own echo!")
        root.resizable(False, False)
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="#0b1021", highlightthickness=0)
        self.canvas.pack()

        self.keys = set()
        self.running = True
        self.game_over = False
        self.win = False
        self.time_accum = 0.0

        # HUD
        self.hud = self.canvas.create_text(
            10, 10, anchor="nw", fill="#cfe3ff",
            font=("Consolas", 14, "bold"),
            text=""
        )

        # Border
        self.canvas.create_rectangle(BORDER, BORDER, WIDTH - BORDER, HEIGHT - BORDER, outline="#415a77")

        # Init game entities
        self.reset_game()

        # Bindings
        self.root.bind("<KeyPress>", self.on_key_down)
        self.root.bind("<KeyRelease>", self.on_key_up)

        # Start loop
        self.tick()

    def reset_game(self):
        self.canvas.delete("entity")
        self.game_over = False
        self.win = False
        self.time_elapsed = 0.0

        # Player start
        self.px, self.py = WIDTH * 0.2, HEIGHT * 0.5
        self.vx, self.vy = 0.0, 0.0
        self.player = self.canvas.create_oval(
            self.px - PLAYER_R, self.py - PLAYER_R, self.px + PLAYER_R, self.py + PLAYER_R,
            fill="#62d2ff", outline="", tags=("entity", "player")
        )

        # Echo setup: record history of player positions; echo lags behind
        delay_frames = max(1, int(ECHO_DELAY_SEC * FPS))
        self.history = deque([(self.px, self.py)] * delay_frames, maxlen=delay_frames)
        hx, hy = self.history[0]
        self.ex, self.ey = hx, hy
        self.echo = self.canvas.create_oval(
            self.ex - ECHO_R, self.ey - ECHO_R, self.ex + ECHO_R, self.ey + ECHO_R,
            fill="#a5adbb", outline="", tags=("entity", "echo")
        )

        # Orbs
        self.orbs = []
        attempts = 0
        while len(self.orbs) < NUM_ORBS and attempts < 1000:
            attempts += 1
            ox = random.randint(BORDER + 30, WIDTH - BORDER - 30)
            oy = random.randint(BORDER + 30, HEIGHT - BORDER - 30)
            # keep away from player start
            if dist2(ox, oy, self.px, self.py) < (PLAYER_R + 120) ** 2:
                continue
            # keep orbs apart
            if any(dist2(ox, oy, x, y) < (ORB_R * 2 + 20) ** 2 for (x, y, _) in self.orbs):
                continue
            orb_id = self.canvas.create_oval(
                ox - ORB_R, oy - ORB_R, ox + ORB_R, oy + ORB_R,
                fill="#5af78e", outline="", tags=("entity", "orb")
            )
            self.orbs.append((ox, oy, orb_id))

        # Enemies (bouncing)
        self.enemies = []
        for _ in range(NUM_ENEMIES):
            ex = random.randint(BORDER + 40, WIDTH - BORDER - 40)
            ey = random.randint(BORDER + 40, HEIGHT - BORDER - 40)
            # avoid immediate spawn kill
            if dist2(ex, ey, self.px, self.py) < (PLAYER_R + 160) ** 2:
                ex = WIDTH * 0.8
                ey = HEIGHT * 0.3
            speed = random.uniform(90, 160)
            angle = random.uniform(0, math.tau)
            evx = math.cos(angle) * speed
            evy = math.sin(angle) * speed
            eid = self.canvas.create_oval(
                ex - ENEMY_R, ey - ENEMY_R, ex + ENEMY_R, ey + ENEMY_R,
                fill="#ff6b6b", outline="", tags=("entity", "enemy")
            )
            self.enemies.append([ex, ey, evx, evy, eid])

        # Floating tip text
        self.tip = self.canvas.create_text(
            WIDTH // 2, HEIGHT - 26,
            fill="#9bbcff", font=("Consolas", 12),
            text="Move: Arrow Keys / WASD   Pause: P   Restart: R   Goal: Collect all orbs, avoid red and your echo.",
            tags=("entity",)
        )

    # ---------------------------
    # Input
    # ---------------------------
    def on_key_down(self, e):
        k = e.keysym.lower()
        self.keys.add(k)
        if k == "p":
            self.running = not self.running
        if k == "r":
            self.reset_game()

    def on_key_up(self, e):
        k = e.keysym.lower()
        self.keys.discard(k)

    def input_vector(self):
        left = ("left" in self.keys) or ("a" in self.keys)
        right = ("right" in self.keys) or ("d" in self.keys)
        up = ("up" in self.keys) or ("w" in self.keys)
        down = ("down" in self.keys) or ("s" in self.keys)
        x = (1 if right else 0) - (1 if left else 0)
        y = (1 if down else 0) - (1 if up else 0)
        if x == 0 and y == 0:
            return 0.0, 0.0
        mag = math.hypot(x, y)
        return x / mag, y / mag

    # ---------------------------
    # Game Loop
    # ---------------------------
    def tick(self):
        if self.running and not self.game_over and not self.win:
            self.update(DT)
        self.root.after(int(1000 / FPS), self.tick)

    def update(self, dt):
        self.time_elapsed += dt

        # Move player
        ix, iy = self.input_vector()
        self.vx = ix * PLAYER_SPEED
        self.vy = iy * PLAYER_SPEED
        self.px += self.vx * dt
        self.py += self.vy * dt

        # Keep inside bounds
        self.px = clamp(self.px, BORDER + PLAYER_R, WIDTH - BORDER - PLAYER_R)
        self.py = clamp(self.py, BORDER + PLAYER_R, HEIGHT - BORDER - PLAYER_R)

        # Update echo position from history
        self.history.append((self.px, self.py))
        hx, hy = self.history[0]
        self.ex, self.ey = hx, hy

        # Move enemies & bounce
        for e in self.enemies:
            e[0] += e[2] * dt
            e[1] += e[3] * dt
            if e[0] <= BORDER + ENEMY_R or e[0] >= WIDTH - BORDER - ENEMY_R:
                e[2] *= -1
                e[0] = clamp(e[0], BORDER + ENEMY_R, WIDTH - BORDER - ENEMY_R)
            if e[1] <= BORDER + ENEMY_R or e[1] >= HEIGHT - BORDER - ENEMY_R:
                e[3] *= -1
                e[1] = clamp(e[1], BORDER + ENEMY_R, HEIGHT - BORDER - ENEMY_R)

        # Collisions: enemies
        for ex, ey, evx, evy, eid in self.enemies:
            if circle_overlap(self.px, self.py, PLAYER_R, ex, ey, ENEMY_R):
                self.lose("You hit an enemy!")
                break

        # Collisions: echo (your past self)
        if not self.game_over and circle_overlap(self.px, self.py, PLAYER_R, self.ex, self.ey, ECHO_R):
            self.lose("You collided with your echo!")

        # Collect orbs
        if not self.game_over:
            remaining = []
            for ox, oy, oid in self.orbs:
                if circle_overlap(self.px, self.py, PLAYER_R, ox, oy, ORB_R):
                    self.canvas.delete(oid)
                else:
                    remaining.append((ox, oy, oid))
            self.orbs = remaining
            if not self.orbs:
                self.win = True
                self.show_banner("You collected all orbs! ✨\nPress R to play again.")

        # Draw updates
        self.redraw()

    def lose(self, reason):
        self.game_over = True
        self.show_banner(f"Game Over 💥\n{reason}\nPress R to restart.")

    def show_banner(self, text):
        overlay = self.canvas.create_rectangle(0, 0, WIDTH, HEIGHT, fill="#000000", stipple="gray50", outline="",
                                               tags=("entity",))
        self.canvas.tag_lower(overlay)
        self.canvas.tag_raise(self.hud)
        self.canvas.create_text(
            WIDTH // 2, HEIGHT // 2, text=text, fill="#ffffff",
            font=("Consolas", 20, "bold"), justify="center", tags=("entity",)
        )

    def redraw(self):
        # Player
        self.canvas.coords(self.player, self.px - PLAYER_R, self.py - PLAYER_R, self.px + PLAYER_R, self.py + PLAYER_R)
        # Echo
        self.canvas.coords(self.echo, self.ex - ECHO_R, self.ey - ECHO_R, self.ex + ECHO_R, self.ey + ECHO_R)
        # Enemies
        for ex, ey, evx, evy, eid in self.enemies:
            self.canvas.coords(eid, ex - ENEMY_R, ey - ENEMY_R, ex + ENEMY_R, ey + ENEMY_R)

        # HUD
        orb_count = len(self.orbs)
        status = "Paused" if not self.running else "Playing"
        if self.game_over:
            status = "Game Over"
        elif self.win:
            status = "You Win!"
        self.canvas.itemconfig(
            self.hud,
            text=f"Status: {status}    Orbs left: {orb_count}    Time: {self.time_elapsed:4.1f}s"
        )


# ---------------------------
# Start Menu
# ---------------------------

class StartMenu:
    def __init__(self, root):
        self.root = root
        root.title("Echo Dash — Start Menu")
        root.resizable(False, False)

        self.frame = tk.Frame(root, width=WIDTH, height=HEIGHT, bg="#0b1021")
        self.frame.pack_propagate(0)
        self.frame.pack()

        title = tk.Label(self.frame, text="Echo Dash", fg="#62d2ff", bg="#0b1021", font=("Consolas", 32, "bold"))
        title.pack(pady=40)

        btn = tk.Button(self.frame, text="Start Game", font=("Consolas", 18, "bold"),
                        bg="#5af78e", fg="#000000", command=self.start_game)
        btn.pack(pady=20)

        info = tk.Label(self.frame,
                        text="Survive your own echo.\nCollect all green orbs.\nAvoid red enemies and your shadow!",
                        fg="#cfe3ff", bg="#0b1021", font=("Consolas", 14))
        info.pack(pady=10)

    def start_game(self):
        self.frame.destroy()
        EchoDash(self.root)


def main():
    root = tk.Tk()
    StartMenu(root)
    root.mainloop()


if __name__ == "__main__":
    main()


In [2]:
import tkinter as tk
import random
import math
from collections import deque

WIDTH, HEIGHT = 800, 560
FPS = 60
DT = 1 / FPS
PLAYER_SPEED = 220
PLAYER_R = 12
ECHO_R = 10
ORB_R = 8
ENEMY_R = 12
NUM_ORBS = 10
NUM_ENEMIES = 5
ECHO_DELAY_SEC = 1.6
BORDER = 12
SAFE_START_TIME = 3.0  # seconds of invulnerability after start


def clamp(v, lo, hi):
    return max(lo, min(hi, v))


def dist2(ax, ay, bx, by):
    dx, dy = ax - bx, ay - by
    return dx * dx + dy * dy


def circle_overlap(ax, ay, ar, bx, by, br):
    return dist2(ax, ay, bx, by) <= (ar + br) ** 2


class EchoDash:
    def __init__(self, root):
        self.root = root
        root.title("Echo Dash — Survive your own echo!")
        root.resizable(False, False)
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="#0b1021", highlightthickness=0)
        self.canvas.pack()

        self.keys = set()
        self.running = True
        self.game_over = False
        self.win = False
        self.time_elapsed = 0.0
        self.safe_time_left = SAFE_START_TIME  # countdown at start

        # HUD
        self.hud = self.canvas.create_text(
            10, 10, anchor="nw", fill="#cfe3ff",
            font=("Consolas", 14, "bold"),
            text=""
        )

        # Border
        self.canvas.create_rectangle(BORDER, BORDER, WIDTH - BORDER, HEIGHT - BORDER, outline="#415a77")

        # Init game entities
        self.reset_game()

        # Bindings
        self.root.bind("<KeyPress>", self.on_key_down)
        self.root.bind("<KeyRelease>", self.on_key_up)

        # Start loop
        self.tick()

    def reset_game(self):
        self.canvas.delete("entity")
        self.game_over = False
        self.win = False
        self.time_elapsed = 0.0
        self.safe_time_left = SAFE_START_TIME

        # Player start
        self.px, self.py = WIDTH * 0.2, HEIGHT * 0.5
        self.vx, self.vy = 0.0, 0.0
        self.player = self.canvas.create_oval(
            self.px - PLAYER_R, self.py - PLAYER_R, self.px + PLAYER_R, self.py + PLAYER_R,
            fill="#62d2ff", outline="", tags=("entity", "player")
        )

        # Echo setup
        delay_frames = max(1, int(ECHO_DELAY_SEC * FPS))
        self.history = deque([(self.px, self.py)] * delay_frames, maxlen=delay_frames)
        hx, hy = self.history[0]
        self.ex, self.ey = hx, hy
        self.echo = self.canvas.create_oval(
            self.ex - ECHO_R, self.ey - ECHO_R, self.ex + ECHO_R, self.ey + ECHO_R,
            fill="#a5adbb", outline="", tags=("entity", "echo")
        )

        # Orbs
        self.orbs = []
        for _ in range(NUM_ORBS):
            ox = random.randint(BORDER + 30, WIDTH - BORDER - 30)
            oy = random.randint(BORDER + 30, HEIGHT - BORDER - 30)
            orb_id = self.canvas.create_oval(
                ox - ORB_R, oy - ORB_R, ox + ORB_R, oy + ORB_R,
                fill="#5af78e", outline="", tags=("entity", "orb")
            )
            self.orbs.append((ox, oy, orb_id))

        # Enemies
        self.enemies = []
        for _ in range(NUM_ENEMIES):
            ex = random.randint(BORDER + 40, WIDTH - BORDER - 40)
            ey = random.randint(BORDER + 40, HEIGHT - BORDER - 40)
            speed = random.uniform(90, 160)
            angle = random.uniform(0, math.tau)
            evx = math.cos(angle) * speed
            evy = math.sin(angle) * speed
            eid = self.canvas.create_oval(
                ex - ENEMY_R, ey - ENEMY_R, ex + ENEMY_R, ey + ENEMY_R,
                fill="#ff6b6b", outline="", tags=("entity", "enemy")
            )
            self.enemies.append([ex, ey, evx, evy, eid])

    # ---------------------------
    # Input
    # ---------------------------
    def on_key_down(self, e):
        k = e.keysym.lower()
        self.keys.add(k)
        if k == "p":
            self.running = not self.running
        if k == "r":
            self.reset_game()

    def on_key_up(self, e):
        k = e.keysym.lower()
        self.keys.discard(k)

    def input_vector(self):
        left = ("left" in self.keys) or ("a" in self.keys)
        right = ("right" in self.keys) or ("d" in self.keys)
        up = ("up" in self.keys) or ("w" in self.keys)
        down = ("down" in self.keys) or ("s" in self.keys)
        x = (1 if right else 0) - (1 if left else 0)
        y = (1 if down else 0) - (1 if up else 0)
        if x == 0 and y == 0:
            return 0.0, 0.0
        mag = math.hypot(x, y)
        return x / mag, y / mag

    # ---------------------------
    # Game Loop
    # ---------------------------
    def tick(self):
        if self.running and not self.game_over and not self.win:
            self.update(DT)
        self.root.after(int(1000 / FPS), self.tick)

    def update(self, dt):
        self.time_elapsed += dt
        if self.safe_time_left > 0:
            self.safe_time_left -= dt
            if self.safe_time_left < 0:
                self.safe_time_left = 0

        # Move player
        ix, iy = self.input_vector()
        self.vx = ix * PLAYER_SPEED
        self.vy = iy * PLAYER_SPEED
        self.px += self.vx * dt
        self.py += self.vy * dt

        # Keep inside bounds
        self.px = clamp(self.px, BORDER + PLAYER_R, WIDTH - BORDER - PLAYER_R)
        self.py = clamp(self.py, BORDER + PLAYER_R, HEIGHT - BORDER - PLAYER_R)

        # Update echo position
        self.history.append((self.px, self.py))
        hx, hy = self.history[0]
        self.ex, self.ey = hx, hy

        # Move enemies
        for e in self.enemies:
            e[0] += e[2] * dt
            e[1] += e[3] * dt
            if e[0] <= BORDER + ENEMY_R or e[0] >= WIDTH - BORDER - ENEMY_R:
                e[2] *= -1
            if e[1] <= BORDER + ENEMY_R or e[1] >= HEIGHT - BORDER - ENEMY_R:
                e[3] *= -1

        # Only check collisions AFTER safe time ends
        if self.safe_time_left == 0:
            # Enemy collision
            for ex, ey, evx, evy, eid in self.enemies:
                if circle_overlap(self.px, self.py, PLAYER_R, ex, ey, ENEMY_R):
                    self.lose("You hit an enemy!")
                    break

            # Echo collision
            if not self.game_over and circle_overlap(self.px, self.py, PLAYER_R, self.ex, self.ey, ECHO_R):
                self.lose("You collided with your echo!")

        # Collect orbs (always active)
        if not self.game_over:
            remaining = []
            for ox, oy, oid in self.orbs:
                if circle_overlap(self.px, self.py, PLAYER_R, ox, oy, ORB_R):
                    self.canvas.delete(oid)
                else:
                    remaining.append((ox, oy, oid))
            self.orbs = remaining
            if not self.orbs:
                self.win = True
                self.show_banner("You collected all orbs! ✨\nPress R to play again.")

        # Draw updates
        self.redraw()

    def lose(self, reason):
        self.game_over = True
        self.show_banner(f"Game Over 💥\n{reason}\nPress R to restart.")

    def show_banner(self, text):
        self.canvas.create_rectangle(0, 0, WIDTH, HEIGHT, fill="#000000", stipple="gray50", outline="",
                                     tags=("entity",))
        self.canvas.create_text(
            WIDTH // 2, HEIGHT // 2, text=text, fill="#ffffff",
            font=("Consolas", 20, "bold"), justify="center", tags=("entity",)
        )

    def redraw(self):
        # Player
        self.canvas.coords(self.player, self.px - PLAYER_R, self.py - PLAYER_R, self.px + PLAYER_R, self.py + PLAYER_R)
        # Echo
        self.canvas.coords(self.echo, self.ex - ECHO_R, self.ey - ECHO_R, self.ex + ECHO_R, self.ey + ECHO_R)
        # Enemies
        for ex, ey, evx, evy, eid in self.enemies:
            self.canvas.coords(eid, ex - ENEMY_R, ey - ENEMY_R, ex + ENEMY_R, ey + ENEMY_R)

        # HUD
        orb_count = len(self.orbs)
        status = "Paused" if not self.running else "Playing"
        if self.game_over:
            status = "Game Over"
        elif self.win:
            status = "You Win!"

        countdown = f" | Starting in {self.safe_time_left:.1f}s" if self.safe_time_left > 0 else ""
        self.canvas.itemconfig(
            self.hud,
            text=f"Status: {status}{countdown}    Orbs left: {orb_count}    Time: {self.time_elapsed:4.1f}s"
        )


# ---------------------------
# Start Menu
# ---------------------------

class StartMenu:
    def __init__(self, root):
        self.root = root
        root.title("Echo Dash — Start Menu")
        root.resizable(False, False)

        self.frame = tk.Frame(root, width=WIDTH, height=HEIGHT, bg="#0b1021")
        self.frame.pack_propagate(0)
        self.frame.pack()

        title = tk.Label(self.frame, text="Echo Dash", fg="#62d2ff", bg="#0b1021", font=("Consolas", 32, "bold"))
        title.pack(pady=40)

        btn = tk.Button(self.frame, text="Start Game", font=("Consolas", 18, "bold"),
                        bg="#5af78e", fg="#000000", command=self.start_game)
        btn.pack(pady=20)

        info = tk.Label(self.frame,
                        text="Survive your own echo.\nCollect all green orbs.\nAvoid red enemies and your shadow!\n"
                             "You get 3 seconds safe time at the start.",
                        fg="#cfe3ff", bg="#0b1021", font=("Consolas", 14))
        info.pack(pady=10)

    def start_game(self):
        self.frame.destroy()
        EchoDash(self.root)


def main():
    root = tk.Tk()
    StartMenu(root)
    root.mainloop()


if __name__ == "__main__":
    main()
